<a href="https://colab.research.google.com/github/yashodhicy/Foodo/blob/chatbot/Foodrate_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#import necessary datafiles
import warnings
warnings.filterwarnings("ignore")
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import numpy as np 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
import random
from keras.models import load_model



In [5]:
# git clone and read training json
words=[]
classes = []
documents = []
ignore_words = ['?', '!']

!git clone "https://github.com/yashodhicy/Foodo.git"

data_file =open("/content/Foodo/trainbot.json").read()
intents = json.loads(data_file)
#print(intents)

Cloning into 'Foodo'...
remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 8 (delta 1), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (8/8), done.


pre processing step

In [6]:
#tokenizing json file
nltk.download('punkt')
nltk.download('wordnet')
for intent in intents['intents']:
  for pattern in intent['patterns']:
#tokenize each word
        w = nltk.word_tokenize(pattern)
        #print(w)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
print(words)
#print(documents)
#print(classes)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['hi', 'how', 'are', 'you', 'is', 'anyone', 'there', 'hello', 'whats', 'up', 'hey', 'yo', 'listen', 'please', 'help', 'me', 'i', 'am', 'learner', 'from', 'i', 'belong', 'to', 'aiml', 'batch', 'aifl', 'batch', 'i', 'am', 'from', 'my', 'pm', 'is', 'blended', 'online', 'i', 'am', 'from', 'hey', 'ya', 'talking', 'to', 'you', 'for', 'first', 'time', 'thank', 'you', 'thanks', 'cya', 'see', 'you', 'later', 'see', 'you', 'later', 'goodbye', 'i', 'am', 'leaving', 'have', 'a', 'Good', 'day', 'you', 'helped', 'me', 'thanks', 'a', 'lot', 'thanks', 'a', 'ton', 'you', 'are', 'the', 'best', 'great', 'help', 'too', 'good', 'you', 'are', 'a', 'good', 'learning', 'buddy', 'olympus', 'explain', 'me', 'how', 'olympus', 'works', 'I', 'am', 'not', 'able', 'to', 'understand', 'olympus', 'olympus', 'window', 'not', 'working', 'no', 'access', 'to', 'olympus', 'unable', 'to', 'see', 'link', 'in', 'olympus', 'no', 'link', 'visible', 'on', 'olympus', 'whom', 'to', 'contact', 'for', 'olympus', 'lot', 'of', 'proble

[nltk_data] Downloading package wordnet to /root/nltk_data...


Remove duplications

In [8]:
#lemmatize, remove duplications, all words to lower case
nltk.download('omw-1.4')

# create an object of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
print(words)
words2 = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# li = []
# li = [item for item in words if item not in words2]
# print(li)

words = sorted(list(set(words)))
print(words)
#  sort calsses
classes = sorted(list(set(classes)))
print(classes)

# documents = combination between patterns and intents
print (len(documents), "documents",documents)

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

# creating a pickle file to store the Python objects which we will use while predicting
pickle.dump(words,open('words.pkl','wb')) 
pickle.dump(classes,open('classes.pkl','wb'))

['Good', 'I', 'ML', 'SL', 'a', 'able', 'access', 'activation', 'ada', 'adam', 'aifl', 'aiml', 'am', 'an', 'ann', 'anyone', 'are', 'artificial', 'backward', 'bad', 'bagging', 'batch', 'bayes', 'belong', 'best', 'blended', 'bloody', 'boosting', 'bot', 'buddy', 'classification', 'contact', 'create', 'cross', 'cya', 'day', 'deep', 'did', 'diffult', 'do', 'ensemble', 'epoch', 'epochs', 'explain', 'first', 'for', 'forest', 'forward', 'from', 'function', 'good', 'goodbye', 'gradient', 'great', 'hate', 'have', 'hell', 'hello', 'help', 'helped', 'hey', 'hi', 'hidden', 'hours', 'how', 'hyper', 'i', 'imputer', 'in', 'intelligence', 'is', 'jerk', 'joke', 'knn', 'later', 'layers', 'learner', 'learning', 'leaving', 'link', 'listen', 'logistic', 'lot', 'machine', 'me', 'my', 'naive', 'name', 'nb', 'nets', 'networks', 'neural', 'no', 'not', 'of', 'olympus', 'olypus', 'on', 'online', 'operation', 'opertions', 'otimizer', 'parameters', 'piece', 'please', 'pm', 'problem', 'problems', 'propagation', 'rand

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Create training data

In [21]:
training = []

# create an empty array for our output
output_emplty = [0] * len(classes)

#training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    #print(pattern_words)
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)
#print(training)
# create train and test lists
train_x = list(training[:,0])
train_y = list(training[:,1])
#print(train_x)
#print(train_y)
print("Training data created")


Training data created


Create NN model to predict the results

In [ ]:
from keras.optimizers import SGD
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot.h5', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully!")


In [33]:
#load the saved model file
model = load_model('chatbot.h5')
intents = json.loads(open("/content/Foodo/trainbot.json").read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

data cleaning

In [34]:
def clean_up_sentence(sentence):

    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):

    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)

    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
               
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
   
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    error = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>error]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [35]:
# function to get the response from the model

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

# function to predict the class and get the response

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

start the chat

In [39]:
# function to start the chat bot which will continue till the user type 'end'

def start_chat():
  print("Bot: This is Yashi! Your Personal Assistant.\n\n")
  while True:
        inp = str(input()).lower()
        if inp.lower()=="end":
            break
        if inp.lower()== '' or inp.lower()== '*':
            print('Please re-phrase your query!')
            print("-"*50)
        else:
            print(f"Bot: {chatbot_response(inp)}"+'\n')
            print("-"*50)

In [40]:
# start the chat bot
start_chat()

Bot: This is Yashi! Your Personal Assistant.


hi
1/1 [==============================] - 0s 86ms/step
Bot: Hello! how can i help you ?

--------------------------------------------------
food
1/1 [==============================] - 0s 19ms/step
Bot: Link: Machine Learning wiki 

--------------------------------------------------
deep learning
1/1 [==============================] - 0s 19ms/step
Bot: Link: Neural Nets wiki

--------------------------------------------------
pissu
1/1 [==============================] - 0s 26ms/step
Bot: Link: Machine Learning wiki 

--------------------------------------------------
bye
1/1 [==============================] - 0s 16ms/step
Bot: Link: Machine Learning wiki 

--------------------------------------------------
ok
1/1 [==============================] - 0s 22ms/step
Bot: Link: Machine Learning wiki 

--------------------------------------------------
thank ou
1/1 [==============================] - 0s 17ms/step
Bot: I hope I was able to assist you